<div style='float: right'><img src='pic/countryroad.png'/></div>
## <div id='countryroad' />カントリーロード

In [ ]:
import numpy as np
from collections import defaultdict
from itertools import chain, groupby, product
from pulp import *
from unionfind import unionfind
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = np.array([list(s) for s in """\
AABBBBC
AADDBEF
GHHDBEF
IHHJKKF
ILMNKKO
ILMNNPP
QMMMMPP""".split()])
nums = {'A':4, 'E':1, 'F':3, 'H':2, 'I':1, 'K':4}
nw, nh = len(data[0]), len(data)

### 問題
* いくつかのマスに線を引き、１つの輪を作ります
* 線は、交差や枝分かれしてはいけません
* 太線で区切られたところ（国と呼ぶ）すべてを１回ずつだけ通ります
* 数字は、その数字がある国を線が通るマス数を表します
* 数字のない国には、何マスでもよいです
* 線が通らないマスが、太線（国境）をはさんでタテヨコに隣接してはいけません

### 変数
* vh：0:L, 1:R (1)
* vv：0:U, 1:D (2)
* vhs (3), vvs (4)

### 制約
* vh,vv,vhs,vvsの関係 (5)
* 入る数と出る数が同じであること (6)
* 交差なし (7)
* 国違いならどちらかに道 (8)
* 指定した数と同じ道数 (9)
* 国に入る数は1 (10)
* 1つの輪 (11)

In [ ]:
m = LpProblem()
vh = np.array(addbinvars(nh, nw+1, 2)) # 0:L, 1:R (1)
vv = np.array(addbinvars(nh+1, nw, 2)) # 0:U, 1:D (2)
vhs = np.array(addvars(nh, nw+1)) # (3)
vvs = np.array(addvars(nh+1, nw)) # (4)
vhs[:,0] = vhs[:,nw] = vvs[0,:] = vvs[nh,:] = 0 # (5)
w = vhs[:,:-1]+vhs[:,1:]+vvs[:-1,:]+vvs[1:,:]
for e1, e2 in chain(zip(vh.sum(2).flat, vhs.flat),
                    zip(vv.sum(2).flat, vvs.flat)):
    m += lpSum(e1) == e2 # (5)
    m += e2 <= 1 # (5)
dc = defaultdict(list)
for i, j in product(range(nh), range(nw)):
    m += lpSum(vv[i+k,j,k]-vv[i+k,j,1-k]+vh[i,j+k,k]
        -vh[i,j+k,1-k] for k in range(2)) == 0 # (6)
    m += w[i,j] <= 2 # (7)
    if i and data[i-1][j] != data[i][j]:
        m += w[i-1,j]+w[i,j] >= 1 # (8)
        dc[data[i-1][j]].append(vv[i][j][0])
        dc[data[i][j]].append(vv[i][j][1])
    if j and data[i][j-1] != data[i][j]:
        m += w[i,j-1]+w[i,j] >= 1 # (8)
        dc[data[i][j-1]].append(vh[i][j][0])
        dc[data[i][j]].append(vh[i][j][1])
for k, d in groupby(sorted(zip(data.flat, w.flat)),
                    lambda x:x[0]):
   if k in nums:
        m += lpSum(c[1] for c in d) == 2*nums[k] # (9)
for d in dc.values():
    m += lpSum(d) == 1 # (10)

In [ ]:
while True:
    %time m.solve()
    rhs = np.vectorize(value)(vhs)
    rvs = np.vectorize(value)(vvs)
    if m.status != 1: break
    u = unionfind(nh * nw)
    for i, j in product(range(nh), range(nw)):
        if i and rvs[i,j]:
            u.unite(j+i*nw-nw, j+i*nw)
        if j and rhs[i,j]:
            u.unite(j-1+i*nw, j+i*nw)
    if sum(1 for gr in u.groups() if len(gr) > 1) == 1:
        break
    m += (lpSum(vh[rhs>0])+lpSum(vv[rvs>0])
        <= (rhs>0).sum() + (rvs>0).sum() - 1) # (11)
r = ((rvs[:-1,:]>0)+2*(rhs[:,:-1]>0)
  +4*(rvs[1:,:]>0)+8*(rhs[:,1:]>0))
print('\n'.join(''.join(s) for s in np.vectorize(
    lambda x: '　12┘4│┐78└─1┌34┼'[x])(r)))